In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import json
import sys

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, mean_squared_error, r2_score

from sklearn.model_selection import (
    KFold,
    ShuffleSplit,
    RepeatedKFold,
    train_test_split,
    ParameterGrid,
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import ElasticNetCV, ElasticNet



from permetrics.regression import RegressionMetric

In [ ]:
proj_dir = Path('../../..')
seed = 1993

# specify the project directory and file paths
geopackage_fn = (
    proj_dir / "data/gis/geopackages/columbia_river_basin.gpkg"
)  

In [ ]:
ml_input_data = pd.read_csv(proj_dir / 'methods/04-ml_development/input_data/ml_input_data.csv')
# ann1_test_set = pd.read_csv('ANN1_test_set.csv').rename(columns={'block2_pred': 'ann1_y_pred'})
# lr1_test_set = pd.read_csv('LR1_test_set.csv').rename(columns={'y_pred': 'lr1_y_pred'})
rfr1_test_set = pd.read_csv('RFR1_test_set.csv').rename(columns={'y_pred': 'rfr1_y_pred'})
rfr2_test_set = pd.read_csv('RFR2_test_set.csv').rename(columns={'y_pred': 'rfr2_y_pred'})

In [ ]:
# test_set = ml_input_data.merge(rfr1_test_set[['Date', 'avg_temp(C)', 'Name', 'rfr1_y_pred']], on=['Date', 'avg_temp(C)', 'Name', ], how='outer')
test_set = rfr1_test_set.merge(rfr2_test_set[['Date', 'avg_temp(C)', 'Name', 'rfr2_y_pred']], on=['Date', 'avg_temp(C)', 'Name', ], how='outer')

lsat_v_insitu = test_set.dropna(subset=['avg_temp(C)', 'WaterTempC'])

In [ ]:
lsat_evaluator = RegressionMetric(lsat_v_insitu['avg_temp(C)'].to_list(), lsat_v_insitu['WaterTempC'].to_list())
rfr1_evaluator = RegressionMetric(rfr1_test_set['avg_temp(C)'].to_list(), rfr1_test_set['rfr1_y_pred'].to_list())
rfr2_evaluator = RegressionMetric(rfr2_test_set['avg_temp(C)'].to_list(), rfr2_test_set['rfr2_y_pred'].to_list())

In [ ]:
list_metrics = ['MAE', 'MSE', 'RMSE', 'R2', 'NSE', 'KGE']

lsat_metrics = lsat_evaluator.get_metrics_by_list_names(list_metrics)
rfr1_metrics = rfr1_evaluator.get_metrics_by_list_names(list_metrics)
rfr2_metrics = rfr2_evaluator.get_metrics_by_list_names(list_metrics)

In [ ]:
test_set.columns

In [ ]:
# scatter plot of the test results
fig, ([ax0, ax1, ax2], [ax4, ax5, ax6]) = plt.subplots(2, 3, figsize=(16, 8))
test_set.plot.scatter(x="avg_temp(C)", y="WaterTempC", ax=ax0, s=0.75)
test_set.plot.scatter(x="avg_temp(C)", y="rfr1_y_pred", ax=ax1, s=0.75)
test_set.plot.scatter(x="avg_temp(C)", y="rfr2_y_pred", ax=ax2, s=0.75)

ax0.plot([0, 30], [0, 30], color="k", linestyle="--")
ax1.plot([0, 30], [0, 30], color="k", linestyle="--")
ax2.plot([0, 30], [0, 30], color="k", linestyle="--")

ax0.set_xlabel("In-situ Water Temperature (C)")
ax1.set_xlabel("In-situ Water Temperature (C)")
ax2.set_xlabel("In-situ Water Temperature (C)")


ax0.set_ylabel("Landsat Water Temperature (C)")
ax1.set_ylabel("Estimated Water Temperature (C)")
ax2.set_ylabel("Estimated Water Temperature (C)")

ax0.set_title("Landsat\n(a)")
ax1.set_title("Random Forest Regression (Variation 1)\n(b)")
ax2.set_title("Random Forest Regression (Variation 2)\n(c)")


ax0.annotate(
    f'MAE: {lsat_metrics["MAE"]:.2f}', xy=(0.05, 0.9), xycoords="axes fraction"
)
ax0.annotate(
    f'RMSE: {lsat_metrics["RMSE"]:.2f}', xy=(0.05, 0.85), xycoords="axes fraction"
)
ax0.annotate(
    f'NSE: {lsat_metrics["NSE"]:.2f}', xy=(0.05, 0.8), xycoords="axes fraction"
)
ax0.annotate(
    f'KGE: {lsat_metrics["KGE"]:.2f}', xy=(0.05, 0.75), xycoords="axes fraction"
)
# ax0.annotate(
#     f'R2: {lsat_metrics["R2"]:.2f}', xy=(0.05, 0.7), xycoords="axes fraction"
# )
# ax0.annotate(
#     f'MSE: {lsat_metrics["MSE"]:.2f}', xy=(0.05, 0.65), xycoords="axes fraction"
# )

ax1.annotate(
    f'MAE: {rfr1_metrics["MAE"]:.2f}', xy=(0.05, 0.9), xycoords="axes fraction"
)
ax1.annotate(
    f'RMSE: {rfr1_metrics["RMSE"]:.2f}', xy=(0.05, 0.85), xycoords="axes fraction"
)
ax1.annotate(
    f'NSE: {rfr1_metrics["NSE"]:.2f}', xy=(0.05, 0.8), xycoords="axes fraction"
)
ax1.annotate(
    f'KGE: {rfr1_metrics["KGE"]:.2f}', xy=(0.05, 0.75), xycoords="axes fraction"
)
# ax1.annotate(
#     f'R2: {lr1_metrics["R2"]:.2f}', xy=(0.05, 0.7), xycoords="axes fraction"
# )
# ax1.annotate(
#     f'MSE: {lr1_metrics["MSE"]:.2f}', xy=(0.05, 0.65), xycoords="axes fraction"
# )

ax2.annotate(
    f'MAE: {rfr2_metrics["MAE"]:.2f}', xy=(0.05, 0.9), xycoords="axes fraction"
)
ax2.annotate(
    f'RMSE: {rfr2_metrics["RMSE"]:.2f}', xy=(0.05, 0.85), xycoords="axes fraction"
)
ax2.annotate(
    f'NSE: {rfr2_metrics["NSE"]:.2f}', xy=(0.05, 0.8), xycoords="axes fraction"
)
ax2.annotate(
    f'KGE: {rfr2_metrics["KGE"]:.2f}', xy=(0.05, 0.75), xycoords="axes fraction"
)
# ax2.annotate(
#     f'R2: {rfr1_metrics["R2"]:.2f}', xy=(0.05, 0.7), xycoords="axes fraction"
# )
# ax2.annotate(
#     f'MSE: {rfr1_metrics["MSE"]:.2f}', xy=(0.05, 0.65), xycoords="axes fraction"
# )

# ax3.annotate(
#     f'MAE: {ann1_metrics["MAE"]:.2f}', xy=(0.05, 0.9), xycoords="axes fraction"
# )
# ax3.annotate(
#     f'RMSE: {ann1_metrics["RMSE"]:.2f}', xy=(0.05, 0.85), xycoords="axes fraction"
# )
# ax3.annotate(
#     f'NSE: {ann1_metrics["NSE"]:.2f}', xy=(0.05, 0.8), xycoords="axes fraction"
# )
# ax3.annotate(
#     f'KGE: {ann1_metrics["KGE"]:.2f}', xy=(0.05, 0.75), xycoords="axes fraction"
# )
# # ax3.annotate(
# #     f'R2: {ann1_metrics["R2"]:.2f}', xy=(0.05, 0.7), xycoords="axes fraction"
# # )
# # ax3.annotate(
# #     f'MSE: {ann1_metrics["MSE"]:.2f}', xy=(0.05, 0.65), xycoords="axes fraction"
# # )

# # histogram of the errors
# bins = np.arange(-20, 20, 0.5)
# ax4.hist(test_set["avg_temp(C)"]-test_set['WaterTempC'], bins=bins);
# ax5.hist(test_set["avg_temp(C)"]-test_set['lr1_y_pred'], bins=bins);
# ax6.hist(test_set["avg_temp(C)"]-test_set['rfr1_y_pred'], bins=bins);
# ax7.hist(test_set["avg_temp(C)"]-test_set['ann1_y_pred'], bins=bins);

# # ax4.set_title("Landsat")
# # ax5.set_title("Linear Regression")
# # ax6.set_title("Random Forest Regression")
# # ax7.set_title("Artificial Neural Network")
# ax4.set_title("(e)")
# ax5.set_title("(f)")
# ax6.set_title("(g)")
# ax7.set_title("(h)")

# ax4.set_xlabel("Error: In-situ - Landsat (C)")
# ax5.set_xlabel("Error: In-situ - Estimated (C)")
# ax6.set_xlabel("Error: In-situ - Estimated (C)")
# ax7.set_xlabel("Error: In-situ - Estimated (C)")

# ax4.set_ylabel("Frequency")
# ax5.set_ylabel("Frequency")
# ax6.set_ylabel("Frequency")
# ax7.set_ylabel("Frequency")

# # annotate with mean and std
# ax4.annotate(
#     f'Mean: {np.mean(test_set["avg_temp(C)"]-test_set["WaterTempC"]):.2f}', xy=(0.05, 0.9), xycoords="axes fraction"
# )
# ax4.annotate(
#     f'Std: {np.std(test_set["avg_temp(C)"]-test_set["WaterTempC"]):.2f}', xy=(0.05, 0.85), xycoords="axes fraction"
# )
# ax5.annotate(
#     f'Mean: {np.mean(test_set["avg_temp(C)"]-test_set["lr1_y_pred"]):.2f}', xy=(0.05, 0.9), xycoords="axes fraction"
# )
# ax5.annotate(
#     f'Std: {np.std(test_set["avg_temp(C)"]-test_set["lr1_y_pred"]):.2f}', xy=(0.05, 0.85), xycoords="axes fraction"
# )
# ax6.annotate(
#     f'Mean: {np.mean(test_set["avg_temp(C)"]-test_set["rfr1_y_pred"]):.2f}', xy=(0.05, 0.9), xycoords="axes fraction"
# )
# ax6.annotate(
#     f'Std: {np.std(test_set["avg_temp(C)"]-test_set["rfr1_y_pred"]):.2f}', xy=(0.05, 0.85), xycoords="axes fraction"
# )
# ax7.annotate(
#     f'Mean: {np.mean(test_set["avg_temp(C)"]-test_set["ann1_y_pred"]):.2f}', xy=(0.05, 0.9), xycoords="axes fraction"
# )
# ax7.annotate(
#     f'Std: {np.std(test_set["avg_temp(C)"]-test_set["ann1_y_pred"]):.2f}', xy=(0.05, 0.85), xycoords="axes fraction"
# )

# fig.tight_layout()
# plt.savefig('model_comparison.png', dpi=300)

In [ ]:
basins = gpd.read_file(geopackage_fn, layer="Basins") # read the layer "Basins" from the geopackage
rivers = gpd.read_file(geopackage_fn, layer="Rivers") # read the layer "Rivers" from the geopackage
reaches = gpd.read_file(geopackage_fn, layer="Reaches") # read the layer "Reaches" from the geopackage

In [ ]:
reaches.rename(columns={'reach_id': 'Name'}, inplace=True)

In [ ]:
# merge the test set with the reaches
test_set = test_set.merge(reaches[['Name', 'DistToUpDam', 'DistToDownDam']], on='Name', how='left')

In [ ]:
corr_df = test_set[['lr1_y_pred', 'rfr1_y_pred', 'ann1_y_pred', 'WaterTempC', 'avg_temp(C)', 'DistToUpDam', 'DistToDownDam', 'LandTempC', 'WidthMean', 'ClimateClass', 'DOY', 'NDVI']].corr()
corr_df.to_csv('correlation.csv')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
test_set.plot.scatter(x="rfr1_y_pred", y="NDVI", ax=ax, s=0.75)

In [ ]:
test_set[['rfr1_y_pred', "NDVI"]].dropna()

In [ ]:
# scatter of predicted vs land temperature 
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
test_set.plot.scatter(x="LandTempC", y="ann1_y_pred", ax=ax[0], s=0.75)
test_set.plot.scatter(x="LandTempC", y="lr1_y_pred", ax=ax[1], s=0.75)
test_set.plot.scatter(x="LandTempC", y="rfr1_y_pred", ax=ax[2], s=0.75)


In [ ]:
# count non nan values
test_set['ann1_y_pred'].count(), test_set['lr1_y_pred'].count(), test_set['rfr1_y_pred'].count()

In [ ]:
test_set['avg_temp(C)'].count()

In [ ]:
# plot a scatter of the error vs distance to upstream dam
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.scatter(test_set['WidthMean'], (test_set["avg_temp(C)"]-test_set['rfr1_y_pred']), s=0.75)

In [ ]:
test_set.columns

In [ ]:
# bin insitu_lsat by average temperature
test_set['DistToUpDam_bin'] = pd.cut(test_set['DistToUpDam'], bins=np.arange(0, 300, 10))
# test_set['DistToUpDam_bin'] = pd.cut(test_set['WidthMean'], bins=np.arange(0, 5000, 120))
test_set['error2'] = np.abs(test_set["avg_temp(C)"]-test_set['rfr2_y_pred'])
test_set['error1'] = np.abs(test_set["avg_temp(C)"]-test_set['rfr1_y_pred'])

# find the mid and max of the bins
test_set['DistToUpDam_bin_mid'] = test_set['DistToUpDam_bin'].apply(lambda x: x.mid)
test_set['DistToUpDam_bin_max'] = test_set['DistToUpDam_bin'].apply(lambda x: x.right)

# find the mean and std of the error
mean_error = test_set.groupby('DistToUpDam_bin_max')['error'].mean()
std_error = test_set.groupby('DistToUpDam_bin_max')['error'].std()


fig, ax = plt.subplots(1, 1, figsize=(15, 6))
# box plot of the error vs distance to upstream dam bin
test_set.boxplot(column='error', by='DistToUpDam_bin_max', ax=ax)
# ax.plot(mean_error.index,mean_error.values, color='r')
ax.set_ylim(0, 10)

fig.suptitle('')
ax.set_title('Boxplot of absolute error vs distance from upstream dam')
ax.set_xlabel('Distance to upstream dam (km)')
ax.set_ylabel('Absolute error: |In-situ - RFR Estimate| (C)')

In [ ]:
test_set[test_set['DistToUpDam'] <50]['error1'].mean()

In [ ]:
mean_error

In [ ]:
a = test_set.groupby('DistToUpDam')['error']
a.mean().plot(color='r')

In [ ]:
# bin insitu_lsat by average temperature
test_set['DistToUpDam_bin'] = pd.cut(test_set['DistToUpDam'], bins=np.arange(0, 300, 10))
# test_set['DistToUpDam_bin'] = pd.cut(test_set['WidthMean'], bins=np.arange(0, 5000, 120))
test_set['error'] = np.abs(test_set["avg_temp(C)"]-test_set['rfr1_y_pred'])

# find the mid and max of the bins
test_set['DistToUpDam_bin_mid'] = test_set['DistToUpDam_bin'].apply(lambda x: x.mid)
test_set['DistToUpDam_bin_max'] = test_set['DistToUpDam_bin'].apply(lambda x: x.right)

# find the mean and std of the error
mean_error = test_set.groupby('DistToUpDam_bin_max')['error'].mean()
std_error = test_set.groupby('DistToUpDam_bin_max')['error'].std()


fig, ax = plt.subplots(1, 1, figsize=(15, 6))
# box plot of the error vs distance to upstream dam bin
test_set.boxplot(column='error', by='DistToUpDam_bin_max', ax=ax)
# ax.plot(mean_error.index,mean_error.values, color='r')
ax.set_ylim(0, 10)

fig.suptitle('')
ax.set_title('Boxplot of absolute error vs distance from upstream dam')
ax.set_xlabel('Distance to upstream dam (km)')
ax.set_ylabel('Absolute error: |In-situ - RFR Estimate| (C)')

In [ ]:
test_set[test_set['WidthMean'] >2000]

In [ ]:
a = test_set.groupby('WidthMean')['error']
a.mean().plot(color='r')

In [ ]:

test_set.groupby('DistToUpDam_bin_max').aggregate({'error': ['mean', 'std']})['error'].reset_index()

In [ ]:
# correlation between errors and distance to upstream dam
test_set[['error', 'DistToUpDam']].corr()

In [ ]:
cutoff = 50

lt50 = test_set[test_set['DistToUpDam'] < cutoff].dropna(subset=['rfr1_y_pred'])
gt50 = test_set[test_set['DistToUpDam'] >= cutoff].dropna(subset=['avg_temp(C)', 'rfr1_y_pred'])

lt50_metrics = RegressionMetric(lt50['avg_temp(C)'].to_list(), lt50['rfr1_y_pred'].to_list()).get_metrics_by_list_names(list_metrics)
gt50_metrics = RegressionMetric(gt50['avg_temp(C)'].to_list(), gt50['rfr1_y_pred'].to_list()).get_metrics_by_list_names(list_metrics)

In [ ]:
gt50_metrics

In [ ]:
lt50_metrics

In [ ]:
test_set['DistToUpDam'].max()

In [ ]:
downstream_analysis = pd.DataFrame(columns=['cutoff_dist',] + [metric + '_lt' for metric in list_metrics] + [metric + '_gt' for metric in list_metrics])
# downstream_analysis

for i, cutoff_dist in enumerate(np.arange(10, 300, 10)):

    # print(f'cutoff_dist: {cutoff_dist}')
    test_set_ = test_set.dropna(subset=['DistToUpDam', 'avg_temp(C)', 'rfr1_y_pred'])
    lt = test_set_[test_set_['DistToUpDam'] < cutoff_dist]
    gt = test_set_[test_set_['DistToUpDam'] >= cutoff_dist]

    # print(gt.head())

    # # print(f'cutoff_dist: {cutoff_dist}')
    lt_metrics = RegressionMetric(lt['avg_temp(C)'].to_list(), lt['rfr1_y_pred'].to_list()).get_metrics_by_list_names(list_metrics)
    gt_metrics = RegressionMetric(gt['avg_temp(C)'].to_list(), gt['rfr1_y_pred'].to_list()).get_metrics_by_list_names(list_metrics)

    downstream_analysis.loc[i] = [cutoff_dist]+[lt_metrics[metric] for metric in list_metrics] + [gt_metrics[metric] for metric in list_metrics]



In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(20, 5))

downstream_analysis_50 = downstream_analysis[downstream_analysis['cutoff_dist'] == 50]

# bar plot of greater than 50 km vs less than 50 kme
ax[0].bar(['<50 km', '>=50 km'], [lt50_metrics['MAE'], gt50_metrics['MAE']])
ax[1].bar(['<50 km', '>=50 km'], [lt50_metrics['RMSE'], gt50_metrics['RMSE']])
ax[2].bar(['<50 km', '>=50 km'], [lt50_metrics['R2'], gt50_metrics['R2']])
ax[3].bar(['<50 km', '>=50 km'], [lt50_metrics['NSE'], gt50_metrics['NSE']])
ax[4].bar(['<50 km', '>=50 km'], [lt50_metrics['KGE'], gt50_metrics['KGE']])

ax[0].set_title('Mean Absolute Error')
ax[1].set_title('Root Mean Squared Error')
ax[2].set_title('R2')
ax[3].set_title('NSE')
ax[4].set_title('KGE')


In [ ]:
lt50_metrics

In [ ]:
gt50_metrics

In [ ]:
list_metrics

In [ ]:
#  for each distance downstream, plot the lt and gt mae as bar plot
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
downstream_analysis.plot.bar(x='cutoff_dist', y=['MAE_lt', 'MAE_gt'], ax=ax, label=['Within x-km', 'Beyond x-km'])
ax.set_xlabel('Distance downstream of dam (km)')
ax.set_ylabel('Mean Absolute Error (C)')
ax.set_title('Mean Absolute Error vs Distance downstream of dam')


In [ ]:
downstream_analysis

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 5))
ax.plot(downstream_analysis['cutoff_dist'], downstream_analysis['MAE_lt'], label='Within x-km')
ax.set_xlabel('Distance downstream of dam (km)')
ax.set_ylabel('Mean Absolute Error (C)')
ax.set_title('Mean Absolute Error of Reaches Within x-km downstream of dam')


In [ ]:
test_set.groupby('DistToUpDam_bin')['error'].mean().plot()

In [ ]:
all = test_set.dropna(subset=['avg_temp(C)', 'rfr1_y_pred'])

all_metrics = RegressionMetric(all['avg_temp(C)'].to_list(), all['rfr1_y_pred'].to_list()).get_metrics_by_list_names(list_metrics)

In [ ]:
all_metrics

In [ ]:
# plot distribution of errors
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.hist(test_set[test_set['DistToUpDam_bin_mid']==5]['error'], bins=np.arange(-5, 5, 0.05));

In [ ]:
# plot distribution of errors
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.hist(test_set[test_set['DistToUpDam_bin_mid']==65]['error'], bins=np.arange(-5, 5, 0.05));